In [3]:
from deepmusic.moduleloader import ModuleLoader
from deepmusic.keyboardcell import KeyboardCell
import deepmusic.songstruct as music
import numpy as np
import tensorflow as tf

In [4]:
def build_network(self):
    input_dim = ModuleLoader.batch_builders.get_module().get_input_dim()
    
    # note data
    with tf.name_scope('placeholder_inputs'):
        self.inputs = [
            tf.placeholder(
            tf.float32,
                [self.batch_size, input_dim],
                name ='input'
            )
        ]
    
    # target 88 key, binary classification problem
    with tf.name_scope('placeholder_targets'):
        self.targets = [
            tf.placeholder(
            tf.int32,  # 0/1
            [self.batch_size],
            name = 'target') 
        ]
    
    # hidden state
    with tf.name_scope ('placeholder_use_prev'):
        self.use_prev = [
            tf.placeholder(
            tf.bool,
            [],
            name = 'use_prev')
        ]
    
    # define the network
    self.loop_processing = ModuleLoader.loop_processings.build_module(self.args)
    def loop_rnn(prev, i):
        next_input = self.loop_processing(prev)
        return tf.cond(self.prev[i], lambda: next_input, lambda: self.inputs[i])
    
    # build seq2seq model
    self.outputs, self.final_state = tf.nn.seq2seq.rnn_decoder(
        decoder_inputs = self.inputs,
        initial_state = None,
        cell = KeyboardCell,
        loop_function = loop_rnn
    ) # because defined in keyboard cell
    
    # training
    # loss function
    loss_fct = tf.nn.seq2seq.sequence_loss(
        self.outputs, 
        self.targets, 
        softmax_loss_function = tf.nn.softmax.cross_entropy_with_logits,
        average_across_timesteps = True,
        average_across_batch = True
    )
    
    # initialize the optimizer, minimize the loss
    opt = tf.train.AdamOptimizer(
        learning_rate = self.current_learning_rate,
        beta1 = 0.9,
        beta2 = 0.999,
        epsilon = 1e-08
    )
    
    self.opt_op = opt.minimize(loss_fct)

In [5]:
build_network()

TypeError: build_network() missing 1 required positional argument: 'self'